<a href="https://colab.research.google.com/github/ViniciusCastillo/Test_Einstein/blob/main/notebooks/Classification_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# importando as bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from math import sqrt
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.dummy import DummyClassifier
from imblearn.over_sampling import SMOTE
import xgboost as xgb
# configurando formato de apresentação de números
pd.options.display.float_format = "{:,.4f}".format

In [18]:
def roc_auc_score_multi (model, X, y): 
  return roc_auc_score(y_true=y, y_score=model.predict_proba(X), multi_class='ovo')

def test_models (X, y, params):
  cv = RepeatedStratifiedKFold(n_splits = 5, n_repeats = 10)
  result_itens = ['rank_test_score','mean_test_score']
  grid_rfc = GridSearchCV(RandomForestClassifier(), params['rfc'], scoring=roc_auc_score_multi, cv=cv).fit(X,y)
  result_itens_rfc = result_itens.copy()
  result_itens_rfc.extend([item for item in pd.DataFrame(grid_rfc.cv_results_).columns.values if "param_" in item])
  results_rfc = pd.DataFrame(grid_rfc.cv_results_)[result_itens_rfc]
  print('\n Best Random Forest: \n', results_rfc.sort_values(by='rank_test_score').iloc[0,1:])
  grid_knc = GridSearchCV(KNeighborsClassifier(), params['knc'], scoring=roc_auc_score_multi, cv=cv).fit(X,y)
  result_itens_knc = result_itens.copy()
  result_itens_knc.extend([item for item in pd.DataFrame(grid_knc.cv_results_).columns.values if "param_" in item])
  results_knc = pd.DataFrame(grid_knc.cv_results_)[result_itens_knc]
  print('\n best K Neighbors: \n', results_knc.sort_values(by='rank_test_score').iloc[0,1:])
  grid_xgbc = GridSearchCV(xgb.XGBClassifier(), 
                          params['xgbc'], scoring=roc_auc_score_multi, cv=cv).fit(X,y)
  result_itens_xgbc = result_itens.copy()
  result_itens_xgbc.extend([item for item in pd.DataFrame(grid_xgbc.cv_results_).columns.values if "param_" in item])
  results_xgbc = pd.DataFrame(grid_xgbc.cv_results_)[result_itens_xgbc]
  print('\n best XGBoost: \n', results_xgbc.sort_values(by='rank_test_score').iloc[0,1:])

  return {'rfc': results_rfc.sort_values(by='rank_test_score'),
          'knc': results_knc.sort_values(by='rank_test_score'),
          'xgbc': results_xgbc.sort_values(by='rank_test_score')
          }

In [10]:
df = pd.read_csv('https://github.com/ViniciusCastillo/Teste_Einstein/raw/main/datasets/data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x1      10000 non-null  float64
 1   x2      10000 non-null  float64
 2   x3      10000 non-null  float64
 3   target  10000 non-null  object 
dtypes: float64(3), object(1)
memory usage: 312.6+ KB


In [ ]:
df.describe(include='all')

In [ ]:
df.target.value_counts().to_frame()

In [ ]:
sns.boxplot(y=df['x1'], x=df['target'])

In [ ]:
sns.boxplot(y=df['x2'], x=df['target'])

In [ ]:
sns.boxplot(y=df['x3'], x=df['target'])

In [ ]:
sns.pairplot(df, hue='target')

In [11]:
df['x13'] = df['x1']/df['x3']
df['x23'] = df['x2']/df['x3']
df['x12'] = df['x1']/df['x2']
df['x1^2'] = df['x1']**2
df['x2^2'] = df['x2']**2
df['x3^2'] = df['x3']**2
df['x1^3'] = df['x1']**3
df['x2^3'] = df['x2']**3
df['x3^3'] = df['x3']**3
df['log(x1-min(x1)+1)'] = (df['x1']-df['x1'].min()+1).transform(np.log)
df['log(x2-min(x2)+1)'] = (df['x2']-df['x2'].min()+1).transform(np.log)
df['log(x3-min(x3)+1)'] = (df['x3']-df['x3'].min()+1).transform(np.log)
df.describe()

,x1,x2,x3,x13,x23,x12,x1^2,x2^2,x3^2,x1^3,x2^3,x3^3,log(x1-min(x1)+1),log(x2-min(x2)+1),log(x3-min(x3)+1)
count,"10,000.0000","10,000.0000","10,000.0000","10,000.0000","10,000.0000","10,000.0000","10,000.0000","10,000.0000","10,000.0000","10,000.0000","10,000.0000","10,000.0000","10,000.0000","10,000.0000","10,000.0000"
mean,42.6696,9.0232,-24.6032,inf,inf,7.2286,"2,475.8802",108.2501,"1,010.8393","161,189.0132","1,614.7412","-45,026.3697",4.0428,2.1484,3.3392
std,25.5978,5.1802,20.1386,NaN,NaN,10.3192,"2,264.3881",130.1708,"1,065.0106","190,488.6713","3,299.2364","59,111.5780",0.4903,0.5457,0.8520
min,-19.5000,0.1700,-59.9800,"-7,408.0000",-635.0000,-5.5574,0.0000,0.0289,0.0000,"-7,414.8750",0.0049,"-215,784.0720",0.0000,0.0000,0.0000
25%,21.6750,5.2600,-41.9200,-2.4707,-0.5224,2.3449,469.8057,27.6676,72.4201,"10,183.0418",145.5316,"-73,665.4459",3.7418,1.8066,2.9476
50%,42.9400,8.0750,-24.5550,-1.2042,-0.2391,4.9164,"1,843.8437",65.2057,602.9480,"79,174.6571",526.5360,"-14,805.3906",4.1501,2.1866,3.5953
75%,64.1350,11.7300,-7.0800,-0.3453,-0.1061,8.8535,"4,113.2983",137.5929,"1,757.2864","263,806.3961","1,613.9647",-354.8949,4.4383,2.5305,3.9871
max,101.5200,43.0800,9.9900,inf,inf,466.2353,"10,306.3104","1,855.8864","3,597.6004","1,046,296.6318","79,951.5861",997.0030,4.8042,3.7821,4.2623


In [12]:
df.loc[df['x3'] == 0]

,x1,x2,x3,target,x13,x23,x12,x1^2,x2^2,x3^2,x1^3,x2^3,x3^3,log(x1-min(x1)+1),log(x2-min(x2)+1),log(x3-min(x3)+1)
5372,45.8800,11.4700,0.0000,low,inf,inf,4.0000,"2,104.9744",131.5609,0.0000,"96,576.2255","1,509.0035",0.0000,4.1954,2.5096,4.1105


In [13]:
df.loc[df['x3'] == 0, 'x13'] = df.loc[df['x13'] != np.inf, 'x13'].max()*2
df.loc[df['x3'] == 0, 'x23'] = df.loc[df['x23'] != np.inf, 'x23'].max()*2
df.describe()

,x1,x2,x3,x13,x23,x12,x1^2,x2^2,x3^2,x1^3,x2^3,x3^3,log(x1-min(x1)+1),log(x2-min(x2)+1),log(x3-min(x3)+1)
count,"10,000.0000","10,000.0000","10,000.0000","10,000.0000","10,000.0000","10,000.0000","10,000.0000","10,000.0000","10,000.0000","10,000.0000","10,000.0000","10,000.0000","10,000.0000","10,000.0000","10,000.0000"
mean,42.6696,9.0232,-24.6032,-0.1279,0.4208,7.2286,"2,475.8802",108.2501,"1,010.8393","161,189.0132","1,614.7412","-45,026.3697",4.0428,2.1484,3.3392
std,25.5978,5.1802,20.1386,182.9124,46.5787,10.3192,"2,264.3881",130.1708,"1,065.0106","190,488.6713","3,299.2364","59,111.5780",0.4903,0.5457,0.8520
min,-19.5000,0.1700,-59.9800,"-7,408.0000",-635.0000,-5.5574,0.0000,0.0289,0.0000,"-7,414.8750",0.0049,"-215,784.0720",0.0000,0.0000,0.0000
25%,21.6750,5.2600,-41.9200,-2.4707,-0.5224,2.3449,469.8057,27.6676,72.4201,"10,183.0418",145.5316,"-73,665.4459",3.7418,1.8066,2.9476
50%,42.9400,8.0750,-24.5550,-1.2042,-0.2391,4.9164,"1,843.8437",65.2057,602.9480,"79,174.6571",526.5360,"-14,805.3906",4.1501,2.1866,3.5953
75%,64.1350,11.7300,-7.0800,-0.3453,-0.1061,8.8535,"4,113.2983",137.5929,"1,757.2864","263,806.3961","1,613.9647",-354.8949,4.4383,2.5305,3.9871
max,101.5200,43.0800,9.9900,"7,430.0000","2,620.0000",466.2353,"10,306.3104","1,855.8864","3,597.6004","1,046,296.6318","79,951.5861",997.0030,4.8042,3.7821,4.2623


In [ ]:
features = ['x1','x2','x3','x13','x23','x12','x1^2','x2^2','x3^2','x1^3','x2^3','x3^3','log(x1-min(x1)+1)','log(x2-min(x2)+1)','log(x3-min(x3)+1)']
X_train, X_test, y_train, y_test = train_test_split(df[features], df['target'], stratify=df['target'], random_state=2134)
X_train_res, y_train_res = SMOTE(random_state=3124).fit_resample(X_train, y_train)
X_train_res = StandardScaler().fit_transform(X_train_res)
dc = DummyClassifier().fit(X_train, y_train)
print('\n', classification_report(y_test, dc.predict(X_test)))
print('\n', roc_auc_score_multi(dc, X_test, y_test))

In [ ]:
params = {'rfc': {'max_depth': [3,5,7,9], 'n_estimators': [50,100,150], 'random_state': [9284]},
          'knc': {'n_neighbors': [1,2,3,4,5,6,7]},
          'xgbc': {'learning_rate': [0.02,0.05,0.1], 'max_depth': [3,4,5], 'gama': [0,0.25,0.5],
                   'subsample': [.5], 'colsample_bytree': [.8], 'seed ': [8347]}
          }
results = test_models(X_, y_, params)

best Random Forest: 
 mean_test_score      0.6740
param_max_depth           8
param_n_estimators      200
Name: 11, dtype: object
best K Neighbors: 
 mean_test_score     0.7158
param_n_neighbors        3
Name: 2, dtype: object


In [ ]:
results['rfc']

In [ ]:
results['knc']

In [ ]:
results['xgbc']